<a href="https://colab.research.google.com/github/basnugroho/thesis/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os

Accessing google drive files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_df = pd.read_excel("/content/gdrive/MyDrive/S2 Informatika/thesis/data/dataset_tweet_isp_kecil.xlsx")
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      308 non-null    float64
 1   tweet   619 non-null    object 
 2   isp     261 non-null    object 
 3   score   619 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 19.5+ KB


In [ ]:
data_df.head()

,id,tweet,isp,score
0,1.374183e+18,"@BiznetHome min cek DM ya, nanyak status konek...",biznet,0
1,1.374166e+18,"@BiznetHome udah min, hbs tweet trs bisa",biznet,1
2,1.374153e+18,min tolong di cek koneksi internet saya @Bizne...,biznet,0
3,1.374139e+18,"@anggajalu Hi @anggajalu, DM mimin ID pelangga...",biznet,0
4,1.374080e+18,"@dwikaputra Hi @dwikaputra, DM mimin ID kamu y...",biznet,0


Preprocessing (formating)

In [ ]:
import re
import string
import unicodedata
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
import pickle

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def clean_text(kata):
    kata = str(kata)
    kata = kata.lower()
    
    # remove hashtag
    # kata = kata.replace('#', '')
    
    # remove punctuation
    # kata = kata.translate(str.maketrans("","",string.punctuation))
    
    #remove extra spaces
    kata = kata.strip()

    # remove space in encoding
    # kata = kata.replace('< ', '<')
    # kata = kata.replace(' >', '>')
    # kata = kata.replace('@ ', '@')
    
    # remove non ascii chars
    new_words = []
    for word in kata:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    kata = "".join(new_words)

    #upper encoded word
    unlower_list = ['<url>', '<product_name>', '<provider_name>', '<user_mention>']
    for word in kata.split():
      if word in unlower_list:
        kata = re.sub(word, word.upper(), kata)
    return kata

#replace string
def find_replace(kata, kalimat):
    list_to_replace = kalimat.split()
    for i,word in enumerate(list_to_replace):
        if kata in word:
            list_to_replace[i] = kata
    return " ".join(list_to_replace)

def remove_stopwords_id(kalimat):    
    # ambil stopword bawaan
    # stop_factory = StopWordRemoverFactory().get_stop_words()
    
    more_stopword = ['nih', 'sih', 'kok', 'hi', 'yah',
                'mohon', 'nya', 'yg', 'wkwk', 'wkwkwk', 'wkwkwkwk']

    # menggabungkan stopword
    #data = stop_factory + more_stopword
    data = stopwords_id + more_stopword

    dictionary = ArrayDictionary(data)
    string = StopWordRemover(dictionary)
    tokens = nltk.tokenize.word_tokenize(string.remove(kalimat))
    return(" ".join(tokens))

In [ ]:
def encode(kata):
    kata = str(kata)
    # encode urls
    kata = re.sub("http*\S+", "<URL>", kata)

    # contain product internet product name like; indihome, biznet, telkomsel
    kata = re.sub("@bizne*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("@indihom*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("@di_cbn*\S+", "<PRODUCT_NAME>", kata)

    # contain hastagh product internet product name like; indihome, biznet, telkomsel
    kata = re.sub("#biznet*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("#indihome*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("#di_cbn*\S+", "<PRODUCT_NAME>", kata)

    kata = re.sub("indi*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("indeh*\S+", "<PRODUCT_NAME>", kata)
    kata = re.sub("indieho*\S+", "<PRODUCT_NAME>", kata)
    
    # contain ISP name like telkom
    kata = re.sub("@telkom*\S+", "<PROVIDER_NAME>", kata)
    kata = re.sub("#telkom*\S+", "<PROVIDER_NAME>", kata)

    # remove full mentions
    kata = re.sub("@\S+", "<USER_MENTION>", kata)

    #
    #kata = re.sub("#*\S+", "<HASHTAG>", kata)
    
    # emoji
    # kata = re.sub("[\u0001f600-\u0001f64f]", "<EMOJI>", kata)

    # non ascii
    re.sub(r'[^\x00-\x7F]+',' ', kata)
    
    return kata

In [ ]:
# build cleaned and encoded dataset
tweets_cleaned = [ clean_text(encode(tweet)) for tweet in data_df['tweet'] ]

cleaned_df = pd.DataFrame(columns=['tweet', 'isp', 'score'])
cleaned_df['tweet'] = tweets_cleaned
cleaned_df['isp'] = data_df['isp']
cleaned_df['score'] = data_df['score']
cleaned_df.head()

,tweet,isp,score
0,"<USER_MENTION> min cek dm ya, nanyak status ko...",biznet,0
1,"<USER_MENTION> udah min, hbs tweet trs bisa",biznet,1
2,min tolong di cek koneksi internet saya <USER_...,biznet,0
3,<USER_MENTION> hi <USER_MENTION> dm mimin id p...,biznet,0
4,<USER_MENTION> hi <USER_MENTION> dm mimin id k...,biznet,0


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt update
!apt install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k
OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,385 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic 

In [ ]:
!pip freeze > '/content/gdrive/MyDrive/S2 Informatika/thesis/requirements.txt'